
### CAN'T SEE THE FOREST, FOR ALL THE TREES

**Laboration**

Syftet med den här laborationen är att lära oss hur vi på ett korrekt sätt kan tune:a hyperparameters samt träna och jämföra prestanda mellan många olika modeller - för ett verklighetstroget problem.

Vi kommer framförallt jobba med trädbaserade modeller - Decision Tree och Random Forest.

Datasetet som vi kommer jobba med och information om denne hittar ni [här](https://archive.ics.uci.edu/dataset/320/student+performance). Ni ska jobba med filen som heter 'student-por.csv'.

**Instruktioner**

Utför varje uppgift så att jag kan köra er notebook rakt-av, uppifrån och ner! Varje uppgift ska lösas på angiven plats.

**Inlämning**

Skicka in hela din notebook på ITHS-portalen, efter att du döpt om den till *laboration_forest_förnamn_efternamn.ipynb*.

**Uppgift 0**

In [1]:
# importera samtliga nödvändiga paket här.
# efterhand kanske du inser att du behöver importera fler paket, lägg då över importerna hit istället

import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split  
from sklearn.metrics import make_scorer, mean_absolute_error

import pandas as pd
import numpy as np

**Uppgift 1**

Läs på om datasetet. Ladda därefter ner den och läs in den till en pandas dataframe. 

Obs: du bör troligtvis behöva *delimiter=';'* som argument till pd.read_csv funktionen.

In [2]:
with open("data/student-por.csv") as f:
    student_por_df = pd.read_csv(f, sep=";")

student_por_df.head(10)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13
5,GP,M,16,U,LE3,T,4,3,services,other,...,5,4,2,1,2,5,6,12,12,13
6,GP,M,16,U,LE3,T,2,2,other,other,...,4,4,4,1,1,3,0,13,12,13
7,GP,F,17,U,GT3,A,4,4,other,teacher,...,4,1,4,1,1,1,2,10,13,13
8,GP,M,15,U,LE3,A,3,2,services,other,...,4,2,2,1,1,1,0,15,16,17
9,GP,M,15,U,GT3,T,3,4,other,other,...,5,5,1,1,1,5,0,12,12,13


**Uppgift 2**

Utför en ordentlig EDA (Exploratory Data Analysis). Rita upp relevanta plottar och kommentera på det du känner är befogat.

In [6]:
# Kollar om datasetet saknar värden eller innehåller felaktiga värden
for col in student_por_df.columns:
    print(f"{col}: {np.sort(student_por_df[col].unique())}")

school: ['GP' 'MS']
sex: ['F' 'M']
age: [15 16 17 18 19 20 21 22]
address: ['R' 'U']
famsize: ['GT3' 'LE3']
Pstatus: ['A' 'T']
Medu: [0 1 2 3 4]
Fedu: [0 1 2 3 4]
Mjob: ['at_home' 'health' 'other' 'services' 'teacher']
Fjob: ['at_home' 'health' 'other' 'services' 'teacher']
reason: ['course' 'home' 'other' 'reputation']
guardian: ['father' 'mother' 'other']
traveltime: [1 2 3 4]
studytime: [1 2 3 4]
failures: [0 1 2 3]
schoolsup: ['no' 'yes']
famsup: ['no' 'yes']
paid: ['no' 'yes']
activities: ['no' 'yes']
nursery: ['no' 'yes']
higher: ['no' 'yes']
internet: ['no' 'yes']
romantic: ['no' 'yes']
famrel: [1 2 3 4 5]
freetime: [1 2 3 4 5]
goout: [1 2 3 4 5]
Dalc: [1 2 3 4 5]
Walc: [1 2 3 4 5]
health: [1 2 3 4 5]
absences: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 18 21 22 24 26 30 32]
G1: [ 0  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
G2: [ 0  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
G3: [ 0  1  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


Kolumnen "failures" innehåller värden som inte stämmer överens med datasetets dukomentation.

> "15 failures - number of past class failures (numeric: n if 1<=n<3, else 4)"

In [9]:
# Kontrollerar hur många av varje värde finns i "failures"-kolumnen
student_por_df["failures"].value_counts()

failures
0    549
1     70
2     16
3     14
Name: count, dtype: int64

Eftersom att majoriteten av studenterna har 0 på "failures" så väljer jag att anta att de ej har fått underkänt på kursen, m.a.o. de har inte gått om kursen. Då värdet 4 inte alls finns i kolumnen antar jag att 3 istället innebär att eleven har fått underkänt 3 eller fler gånger. (Även i datasetet student-mat.csv så finns endast värden 0-3 i "failures"-kolumnen.)

Förutom detta så saknas inga värden och alla värden håller sig till dokumentationen av datasetet.

**Uppgift 3**

Gör en första hypotes på vilka features du tror är relevanta. 

Tvätta sedan datan och utför även den feature engineering du anser är lämplig. 

Kom ihåg att du även behöver hantera kategoriska kolumner. 

Motivera samtliga beslut och åtgärder med kommentarer.

In [ ]:
# lös uppgift 3 här

**Uppgift 4**

Särkilj nu på features och target. Vår target är kolumnen G3.

Lägg alla features i variabeln X och din target i variabel y.

Obs! Du får inte använda kolumnerna G1 eller G2 som features, det är nästintill fuskig (varför?). Säkerställ att de **inte** är med i dina features.

Spara även namnen på kolumnerna du använder som features i en variabel som du kallar feature_column_names

In [ ]:
# lös uppgift 4 här

**Uppgift 5**

Utför en helt vanlig train/test split. Sätt test size till 20%. Träna sedan en RandomForestRegressor på din träningsdata.

Använd random_state = 42 i både train/test split samt som argument till din modell.

In [ ]:
# lös uppgift 5 här


model = 
model.fit()

Kör nu koden nedan för att hur relativt viktig varje kolumn var för vår träning! Ett högre värde = viktigare kolumn.

Hur tolkar du det du ser? Does it make sense?

In [ ]:
plt.bar(feature_column_names, model.feature_importances_);
plt.xticks(rotation=45);
plt.show()

**Uppgift 6**

Bra. Nu drar vi igång på riktigt.

Kör nu koden nedan för att utföra en GridSearch (med cross-validation) över alla kombinationer av de angivna hyperparameters.

Observera att vi använder hela datasetet X och y här, inte dina splits ovan. GridSearchCV skapar automatiskt splits åt oss.

Dokumentationen för GridSearchCV hittar du [här](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

In [ ]:
# Skapa en parametergrid att söka bland

param_grid = {
    'max_depth': [2, 4, 6],
    'n_estimators': [2, 4, 6, 8, 10]
}

# Initiera en basmodell
rf = RandomForestRegressor()

# Defiiniera metric att räkna performance
score = make_scorer(mean_absolute_error)

# Definiera GridSearch-objektet
grid_search = GridSearchCV(estimator = rf,                # ange basmodell
                           param_grid = param_grid, 
                           cv = 3,                        # 3-fold cross-validation
                           n_jobs = -1,                   # använd alla möjliga processer för snabbare träning
                           verbose = 2,                   # läs i dokumentationen vad detta innebär
                           scoring=score) 

# Utför grid search över alla möjliga kombinationer av dina hyperparameters
grid_search.fit(X, y)

**Uppgift 7**

Kör koden nedan för att utforska vilken information som nu sparats från din GridSearch

In [ ]:
cv_results = grid_search.cv_results_

cv_results.keys()

**Uppgift 8**

Vilken kombination av hyperparameters gav upphov till bäst/sämst performance?

Vilken är den bästa/sämsta scoren?

In [ ]:
# lös uppgift 8 här

**Uppgift 9**

Är den bästa scoren "bra" i vårt sammanhang? Varför? Motivera.

*tips:* utför en ordentlig error analysis.

In [ ]:
# lös uppgift 9 här

**Uppgift 10**

Testa nu att träna med dina egenvalda hyperparameters! 

Du kan lägga till fler värden för max_depth och n_estimators, men du kan också välja fler hyperparameters att tune:a.

Random Forest har många fler hyperparameters, kolla in den officiella dokumentationen [här](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html).

Lägg till minst 4 fler hyperparameters, utöver max_depth och n_estimators, och välj vilka värden på dessa du vill testa. 

Kör igenom en till GridSearch över dessa. Får du bättre resultat än innan?

**Tips**: För din egen skull, försök även första vad de olika hyperparameters du väljer faktiskt kontrollerar.

In [ ]:
# lös uppgift 10 här

**Uppgift 11**

Kör nu även en GridSearch på en enstaka Decision Tree (inte Random Forest) där du testar olika värden på max_depth.

Hur förhåller sig resultatet här mot det vi fick med Random Forest?

Kolla gärna dokumentationen för Decision Tree [här](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html), och se om du kan leka med fler hyperparameters än enbart max_depth.

In [ ]:
# lös uppgift 11 här

**Uppgift 12**


Vi har nu haft ett fixerat dataset och testa massa olika hyperparameters på framförallt Random Forest. 

Men, nu ska vi även börja fiffla med att testa olika features också! 

Kom ihåg - för varje kombination av features får vi GridSearcha om och hitta de hyperparameters som presterar bäst.

Testa nu annorlunda input features och kör en GridSearch över hyperparametrar för Random Forest, och försök hitta den kombination av hyperparameters för Random Forest som presterar bäst på dessa. 

In [ ]:
# lös uppgift 12 här

**Uppgift 13**

Kan du hitta en minsta mängd features som räcker för att få hög prestanda? Dvs, hur många & vilka features kan du drop:a utan någon vidare prestandaförlust?

In [ ]:
# lös uppgift 13 här